In [1]:
import pandas as pd
import numpy as np
from prog.tools import *
from prog.model import *
import warnings, pickle, os, json
warnings.filterwarnings("ignore")

aluminum_division = 12
random_state = 99

In [2]:
input_ = {
    "work_id": "julietest",
    "op": 1,
    "model_id": "20230807120000000",
    "speed": "590",
    "l_angle_ori": "200",
    "l_weight_ori": "200",
    "f_angle_ori": "200",
    "f_weight_ori": "200",
    "material": "aluminum", # 'aluminum'
}

In [3]:
work_id      = input_["work_id"]
op           = input_["op"]
model_id     = input_["model_id"]
speed        = int(input_["speed"])
l_angle_ori  = int(input_["l_angle_ori"])
l_weight_ori = float(input_["l_weight_ori"])
f_angle_ori  = int(input_["f_angle_ori"])
f_weight_ori = float(input_["f_weight_ori"])
material     = input_["material"]

In [4]:
root = r"C:\Users\tzuli\Documents\python\teco"

# 取得predict位置
pred_path = os.path.join(root, "data", "predict")        
os.makedirs(pred_path, exist_ok = True)
output_json = os.path.join(pred_path, "output.json")

# 取得model位置     
model_detail = os.path.join(root, "data", "train", model_id, "model")

config_path = os.path.join(root, "prog", "config.json")
with open(config_path) as f:
    config = json.load(f)

### Load model

In [5]:
side = "L"
model_l = pickle.load(open(os.path.join(model_detail, f"{side}_model.pkl"), "rb"))

side = "F"
model_f = pickle.load(open(os.path.join(model_detail, f"{side}_model.pkl"), "rb"))

In [6]:
df_l = pd.DataFrame([[l_angle_ori, l_weight_ori]], columns = ["初始_L側角度", "初始_L側不平衡量"])
df_f = pd.DataFrame([[f_angle_ori, f_weight_ori]], columns = ["初始_F側角度", "初始_F側不平衡量"])

In [7]:
df_l

,初始_L側角度,初始_L側不平衡量
0,200,200.0


In [8]:
df_f

,初始_F側角度,初始_F側不平衡量
0,200,200.0


In [9]:
angle_init = np.linspace(0, 360, (aluminum_division + 1)).astype(int)
print(f"angle_init = {angle_init}")

df_l = calculate_angle_proportion(df_l, angle_init, aluminum_division, side = "L")            
df_f = calculate_angle_proportion(df_f, angle_init, aluminum_division, side = "F")

angle_init = [  0  30  60  90 120 150 180 210 240 270 300 330 360]


In [10]:
df_l = calculate_weight(df_l, side = "L")
df_f = calculate_weight(df_f, side = "F")

In [11]:
df_l.head()

,初始_L側角度,初始_L側不平衡量,angle_proportion,angle_0,angle_target,3,7,37_delta,3_solution,7_solution,5,5_delta,5_solution,10,10_delta,10_solution
0,200,200.0,0.333333,0.0,55,60.0,140.0,0.0,{'30': '2'},"{'20': '1', '30': '4'}",100.0,0.0,"{'10': '1', '30': '3'}",200.0,0.0,"{'20': '1', '30': '6'}"


In [12]:
num = 8
num*3/10, num*7/10

(2.4, 5.6)

In [13]:
df_f.head()

,初始_F側角度,初始_F側不平衡量,angle_proportion,angle_0,angle_target,3,7,37_delta,3_solution,7_solution,5,5_delta,5_solution,10,10_delta,10_solution
0,200,200.0,0.333333,0.0,55,60.0,140.0,0.0,{'30': '2'},"{'20': '1', '30': '4'}",100.0,0.0,"{'10': '1', '30': '3'}",200.0,0.0,"{'20': '1', '30': '6'}"


In [14]:
df_l1 = encoding(df_l, side = "L")
df_f1 = encoding(df_f, side = "F")

In [15]:
X_l, y_l = split_data(df_l1, random_state, predict = True)
X_f, y_f = split_data(df_f1, random_state, predict = True)

In [16]:
X_l.head()

,angle_proportion,angle_0,3,7,37_delta,5,5_delta,10,10_delta
0,0.333333,0.0,0.3,0.7,0.0,0.5,0.0,1.0,0.0


In [17]:
X_f.head()

,angle_proportion,angle_0,3,7,37_delta,5,5_delta,10,10_delta
0,0.333333,0.0,0.3,0.7,0.0,0.5,0.0,1.0,0.0


In [18]:
y_pred_l = model_l.predict(X_l)[0]
y_pred_f = model_f.predict(X_f)[0]
print(f"real - l: {y_l.values[0]}, f: {y_f.values[0]}")
print(f"pred - l: {y_pred_l}, f: {y_pred_f}")

real - l: 1, f: 1
pred - l: 1, f: 1


In [19]:
astype_feats = ['3', '7', '5', '10']
df_l[astype_feats] = df_l[astype_feats].astype(int).astype(str)
df_f[astype_feats] = df_f[astype_feats].astype(int).astype(str)
l_angle_ori = str(l_angle_ori)
f_angle_ori = str(f_angle_ori)

In [20]:
# 最佳補償值
predict = [
        {
            "l_angle_pred":  l_angle_ori,
            "l_weight_pred": df_l.loc[0, "10"],
            "f_angle_pred":  f_angle_ori,
            "f_weight_pred": df_f.loc[0, "10"]
        }
]
predict

[{'l_angle_pred': '200',
  'l_weight_pred': '200',
  'f_angle_pred': '200',
  'f_weight_pred': '200'}]

In [21]:
# 組合
comb = {
    df_l.loc[0, "10"]: df_l.loc[0, "10_solution"],
    df_f.loc[0, "10"]: df_f.loc[0, "10_solution"]
}
comb

{'200': {'20': '1', '30': '6'}}

In [22]:
predict.append({})

# F側補償值組合
if y_pred_l == 0:
    predict[1].update({
        "l_angle_pred":  l_angle_ori,
        "l_weight_pred": [df_l.loc[0, "3"], df_l.loc[0, "7"]],
    })
    comb.update({df_l.loc[0, "3"]: df_l.loc[0, "3_solution"]})
    comb.update({df_l.loc[0, "7"]: df_l.loc[0, "7_solution"]})

elif y_pred_l == 1:
    predict[1].update({
        "l_angle_pred":  l_angle_ori,
        "l_weight_pred": [df_l.loc[0, "5"], df_l.loc[0, "5"]],
    })
    comb.update({df_l.loc[0, "5"]: df_l.loc[0, "5_solution"]})

# L側補償值組合
if y_pred_f == 0: 
    predict[1].update({
        "f_angle_pred":  f_angle_ori,
        "f_weight_pred": [df_f.loc[0, "3"], df_f.loc[0, "7"]],
    })
    comb.update({df_f.loc[0, "3"]: df_f.loc[0, "3_solution"]})
    comb.update({df_f.loc[0, "7"]: df_f.loc[0, "7_solution"]})

elif y_pred_f == 1:
    predict[1].update({
        "f_angle_pred":  f_angle_ori,
        "f_weight_pred": [df_f.loc[0, "5"], df_f.loc[0, "5"]],
    })
    comb.update({df_f.loc[0, "5"]: df_f.loc[0, "5_solution"]})

predict

[{'l_angle_pred': '200',
  'l_weight_pred': '200',
  'f_angle_pred': '200',
  'f_weight_pred': '200'},
 {'l_angle_pred': '200',
  'l_weight_pred': ['100', '100'],
  'f_angle_pred': '200',
  'f_weight_pred': ['100', '100']}]

In [23]:
result = {
    "status":   "success",
    "reason":   "",
    "work_id":  work_id,
    "op":       op,
    "model_id": model_id,
    "predict":  predict,
    "comb":     comb
}
result

{'status': 'success',
 'reason': '',
 'work_id': 'julietest',
 'op': 1,
 'model_id': '20230807120000000',
 'predict': [{'l_angle_pred': '200',
   'l_weight_pred': '200',
   'f_angle_pred': '200',
   'f_weight_pred': '200'},
  {'l_angle_pred': '200',
   'l_weight_pred': ['100', '100'],
   'f_angle_pred': '200',
   'f_weight_pred': ['100', '100']}],
 'comb': {'200': {'20': '1', '30': '6'}, '100': {'10': '1', '30': '3'}}}